# Импорт библиотек

In [1]:
import re
import string
import warnings
import pickle

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import Word2Vec

nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)

warnings.filterwarnings("ignore")

# Чтение датасета

In [2]:
df = pd.read_csv("dataset.csv")
df.head()

,Unnamed: 0,annotation,GRNTI
0,0,Рассматриваются аспекты и параметры алгоритма ...,50. Автоматика. Вычислительная техника
1,1,Преодолена итерационная процедура решения тран...,73. Транспорт
2,2,Статья посвящена рассмотрению методов управлен...,06. Экономика. Экономические науки
3,3,Впервые используется теорема Геделя о неполнот...,27.43. Математика / Теория вероятностей и мате...
4,4,В 1880-1892 гг. в Якутской области в политичес...,03.61. История. Исторические науки / Этнографи...


# Инициализация функций предобработки

In [3]:
# Приведение к нижнему регистру и очистка текста с помощью регулярных выражений
def preprocess(text):
    text = text.lower()
    text = text.strip()
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text


# Очимстка от стоп-слов
def stopword(string):
    a = [i for i in string.split() if i not in stopwords.words('russian')]
    return ' '.join(a)


# Лемматизация
wl = WordNetLemmatizer()


# Костыль для лемматизации
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


# Токенизация
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string))
    a = [wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)]
    return " ".join(a)


# Функция финальной предобработки
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

# Предобработка текста

In [4]:
df['clean_text'] = df['annotation'].apply(lambda x: finalpreprocess(x))
df.head()

,Unnamed: 0,annotation,GRNTI,clean_text
0,0,Рассматриваются аспекты и параметры алгоритма ...,50. Автоматика. Вычислительная техника,рассматриваются аспекты параметры алгоритма по...
1,1,Преодолена итерационная процедура решения тран...,73. Транспорт,преодолена итерационная процедура решения тран...
2,2,Статья посвящена рассмотрению методов управлен...,06. Экономика. Экономические науки,статья посвящена рассмотрению методов управлен...
3,3,Впервые используется теорема Геделя о неполнот...,27.43. Математика / Теория вероятностей и мате...,впервые используется теорема геделя неполноте ...
4,4,В 1880-1892 гг. в Якутской области в политичес...,03.61. История. Исторические науки / Этнографи...,гг якутской области политической ссылке побыва...


# Подготовка к обучению

In [5]:
# Разделение выборки на обучающую и тестовую
X = df['clean_text']
y = df['GRNTI'].transform(lambda x: int(x[:2]))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [6]:
# Токенизация
X_train_tok = [nltk.word_tokenize(i) for i in X_train]
X_test_tok = [nltk.word_tokenize(i) for i in X_test]

# Векторизация методом Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)

# Подготовленные векторизованные данные
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

# Класс векторизации данных
class MeanEmbeddingVectorizer:
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

# Токенизация данных
df['clean_text_tok'] = [nltk.word_tokenize(i) for i in df['clean_text']]
# Векторизация методом Word2Vec
model = Word2Vec(df['clean_text_tok'], min_count=1)
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))
modelw2v = MeanEmbeddingVectorizer(w2v)

# Подготовленные векторизованные данные
X_train_vectors_w2v = modelw2v.transform(X_train_tok)
X_val_vectors_w2v = modelw2v.transform(X_test_tok)

# Обучение и тестирование моделей

## LogisticRegression

### Предобработка Tf-Idf

In [7]:
lr_tfidf = LogisticRegression(solver='liblinear', C=10, penalty='l2', n_jobs=-1)
lr_tfidf.fit(X_train_vectors_tfidf, y_train)

y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         3
           6       0.30      0.89      0.45        18
          10       0.33      0.20      0.25         5
          11       0.00      0.00      0.00         2
          13       0.33      0.50      0.40         2
          14       0.44      0.64      0.52        11
          16       1.00      0.50      0.67         2
          17       0.00      0.00      0.00         5
          19       0.00      0.00      0.00         1
          27       0.92      0.79      0.85        14
          29       0.00      0.00      0.00         2
          30       0.00      0.00      0.00         1
          31       0.60      0.43      0.50         7
          34       0.22      0.71      0.33         7
          36       0.00      0.00      0.00         1
          37       0.00    

### Предобработка Word2Vec

In [8]:
lr_w2v = LogisticRegression(solver='liblinear', C=10, penalty='l2', n_jobs=-1)
lr_w2v.fit(X_train_vectors_w2v, y_train)

y_predict = lr_w2v.predict(X_val_vectors_w2v)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         3
           6       0.14      1.00      0.24        18
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        11
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         5
          19       0.00      0.00      0.00         1
          27       0.00      0.00      0.00        14
          29       0.00      0.00      0.00         2
          30       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         7
          34       0.00      0.00      0.00         7
          36       0.00      0.00      0.00         1
          37       0.00    

## SGDClassifier

### Предобработка Tf-Idf

In [9]:
sgd_tfidf = LogisticRegression(n_jobs=-1)
sgd_tfidf.fit(X_train_vectors_tfidf, y_train)

y_predict = sgd_tfidf.predict(X_test_vectors_tfidf)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         3
           6       0.18      0.94      0.31        18
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.67      0.18      0.29        11
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         5
          19       0.00      0.00      0.00         1
          27       1.00      0.29      0.44        14
          29       0.00      0.00      0.00         2
          30       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         7
          34       0.20      0.71      0.31         7
          36       0.00      0.00      0.00         1
          37       0.00    

### Предобработка Word2Vec

In [10]:
sgd_w2v = LogisticRegression(n_jobs=-1)
sgd_w2v.fit(X_train_vectors_w2v, y_train)

y_predict = sgd_w2v.predict(X_val_vectors_w2v)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         3
           6       0.14      1.00      0.24        18
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        11
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         5
          19       0.00      0.00      0.00         1
          27       0.00      0.00      0.00        14
          29       0.00      0.00      0.00         2
          30       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         7
          34       0.00      0.00      0.00         7
          36       0.00      0.00      0.00         1
          37       0.00    

## DecisionTreeClassifier

### Предобработка Tf-Idf

In [11]:
dt_tfidf = DecisionTreeClassifier()
dt_tfidf.fit(X_train_vectors_tfidf, y_train)

y_predict = dt_tfidf.predict(X_test_vectors_tfidf)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.40      0.67      0.50         3
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         3
           6       0.38      0.28      0.32        18
          10       0.29      0.40      0.33         5
          11       0.00      0.00      0.00         2
          13       0.25      0.50      0.33         2
          14       0.23      0.27      0.25        11
          16       0.50      0.50      0.50         2
          17       1.00      0.20      0.33         5
          19       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         0
          27       0.75      0.21      0.33        14
          29       0.33      0.50      0.40         2
          30       0.00      0.00      0.00         1
          31       0.62      0.71      0.67         7
          34       0.22    

### Предобработка Word2Vec

In [12]:
dt_w2v = DecisionTreeClassifier()
dt_w2v.fit(X_train_vectors_w2v, y_train)

y_predict = dt_w2v.predict(X_val_vectors_w2v)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.40      0.67      0.50         3
           2       0.00      0.00      0.00         0
           3       0.20      0.25      0.22         4
           4       0.00      0.00      0.00         3
           6       0.17      0.11      0.13        18
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.29      0.18      0.22        11
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         5
          19       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         0
          27       0.00      0.00      0.00        14
          29       0.00      0.00      0.00         2
          30       0.00      0.00      0.00         1
          31       0.00    

## RandomForestClassifier

### Предобработка Tf-Idf

In [13]:
rf_tfidf = RandomForestClassifier(n_jobs=-1)
rf_tfidf.fit(X_train_vectors_tfidf, y_train)

y_predict = rf_tfidf.predict(X_test_vectors_tfidf)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         3
           6       0.23      0.83      0.36        18
          10       0.33      0.40      0.36         5
          11       0.00      0.00      0.00         2
          13       0.50      0.50      0.50         2
          14       0.27      0.36      0.31        11
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         5
          19       0.00      0.00      0.00         1
          27       0.89      0.57      0.70        14
          29       0.00      0.00      0.00         2
          30       0.00      0.00      0.00         1
          31       1.00      0.29      0.44         7
          34       0.29      0.57      0.38         7
          36       0.00    

### Предобработка Word2Vec

In [14]:
rf_w2v = RandomForestClassifier(n_jobs=-1)
rf_w2v.fit(X_train_vectors_w2v, y_train)

y_predict = rf_w2v.predict(X_val_vectors_w2v)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           3       0.50      0.25      0.33         4
           4       0.00      0.00      0.00         3
           6       0.23      0.72      0.35        18
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         2
          13       0.50      0.50      0.50         2
          14       0.22      0.18      0.20        11
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         5
          19       0.00      0.00      0.00         1
          27       0.25      0.07      0.11        14
          29       0.00      0.00      0.00         2
          30       0.00      0.00      0.00         1
          31       0.50      0.14      0.22         7
          34       0.09      0.43      0.15         7
          36       0.00      0.00      0.00         1
          37       0.00    

## MultinomialNB

### Предобработка Tf-Idf

In [15]:
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)

y_predict = nb_tfidf.predict(X_test_vectors_tfidf)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         3
           6       0.16      1.00      0.27        18
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        11
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         5
          19       0.00      0.00      0.00         1
          27       1.00      0.07      0.13        14
          29       0.00      0.00      0.00         2
          30       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         7
          34       0.36      0.71      0.48         7
          36       0.00      0.00      0.00         1
          37       0.00    

# Сохранение модели

In [16]:
# Сохранение модели
pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr_tfidf, file)

# Загрузка модели
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)

model

LogisticRegression(C=10, n_jobs=-1, solver='liblinear')